# Script para raspar dados do site da FGV
Esse script usa os módulos Requests e BeautifulSoup para salvar as informações deste [link](http://direitosp.fgv.br/home-professor/A/all/all) em uma tabela .csv

## Importando os módulos necessários
- **Requests** para acessar a página
- **BeatifulSoup** para copiar os elementos da página
- **Pandas** para organizar a informação em uma tabela
- **Time** para colocar intervalos entre cada solicitação e não sobrecarregar o site

In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time


## Raspando os dados

Antes de definir o raspador, precisamos criar um lugar para guardar as informações que salvamos.

In [66]:
# professores é uma lista. Dentro dessa lista, vamos colocar um dicionário com os dados de cada um deles.
professores = []

Note que são 38 páginas de dados e que a url muda de acordo com a página para o seguinte formato: */all/all/all?page=[NÚMERO_DA_PÁGINA]*

É possível, então, criar um loop que altera a url de acordo com a página que você quer extrair as informações.

É dentro desse loop que o raspador vai operar.

In [71]:
# Ei, Python! Para cada número entre 0 e 38 - o 39, limite superior, não entra na conta -, repita os seguintes passos:
for page in range(0,39):
    # Aguarde um segundo para não sobrecarregar o site da FGV
    time.sleep(1)
    # Atualize a URL com o número de cada página
    url = "http://direitosp.fgv.br/home-professor/all/all/all?page=" + str(page) # str(page) trasnforma o número em texto
    # Acesse o endereço que guardamos na URL
    response = requests.get(url)
    # Passe tudo que você encontrar para o raspador
    doc = BeautifulSoup(response.text, 'html.parser')
    # Procure todos os elementos 'div' com a classe "views-field-title" - é lá onde está o nome do professor
    divs = doc.findAll("div", { "class" : "views-field-title" })
    # Para cada um dos 'divs' que achamos, encontre o 'a' - é lá que estão o hiperlink e o nome do professor
    for div in divs:
        # Crie um dicionário para o professor
        professor = {}
        # Acesse a tag 'a', onde estão guardado
        data = div.find("a")
        # Acessa o link e salva na categoria "link" do dicionário
        professor["link"] = "http://direitosp.fgv.br" + data['href']
        # Acessa o link e salva na categoria "nome" do dicionário
        professor["nome"] = data.text
        # Agora, vamos passar para o próximo elemento div, onde está salva a descrição
        # Isso funciona porque os dois dados que quero estão um em cima do outro
        # Note que vou usar duas vezes o método .next_sibling para 'pular' uma linha em branco que separa os dois elementos
        data = div.next_sibling.next_sibling
        # Acessa o texto e salva na categoria "decsrição" do dicionário"
        professor["descricao"] = data.text.strip()
        # Agora que estamos com o dicionário pronto, só preciso adicionar a entrada 'professor' na lista que definimos lá em cima
        professores.append(professor)

### Salvando como csv
Aqui, uso o módulo Pandas para transformar a lista de dicionários lá em cima em um dataframe - parecido com uma plainlha de Excel. Depois, salvo para um arquivo .csv com o encoding adequado.

In [74]:
# ransforme em uma tabela
df = pd.DataFrame(professores)
# Salve em .csv
df.to_csv("professores_fgv.csv",sep="|",encoding="UTF-16")